In [23]:
import os
import glob
import numpy as np
import pandas as pd
import nibabel as nib
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.utils import Sequence
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Dropout, BatchNormalization, Activation, Multiply, Add
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tqdm import tqdm # For progress bars

# --- Config ---
TRAIN_DIR = '/kaggle/input/instant-odc-ai-hackathon/Train'
TEST_DIR = '/kaggle/input/instant-odc-ai-hackathon/test' 
SAVE_DIR = '/kaggle/working/processed_data' # Intermediate storage for speed
IMG_SIZE = 128
BATCH_SIZE = 64  # Increased batch size because 2D is lighter
N_CLASSES = 4
SLICE_DECIMATION = 2 # Skip every 2nd slice to save disk space

In [24]:
# Create directories
os.makedirs(f'{SAVE_DIR}/images', exist_ok=True)
os.makedirs(f'{SAVE_DIR}/masks', exist_ok=True)

def normalize(volume):
    mask = volume > 0
    mean = volume[mask].mean() if np.any(mask) else 0
    std = volume[mask].std() if np.any(mask) else 1
    if std == 0: return volume
    return (volume - mean) / std

def process_and_save_patient(case_path, case_id):
    def get_path(pattern):
        matches = glob.glob(os.path.join(case_path, f"*{pattern}*"))
        return matches[0] if matches else None

    paths = [get_path("flair"), get_path("t1ce"), get_path("t1"), get_path("t2")]
    seg_path = get_path("seg")

    if not all(paths) or not seg_path: return

    try:
        # Load 3D Volume
        X_vol = np.stack([nib.load(p).get_fdata() for p in paths], axis=-1)
        y_vol = nib.load(seg_path).get_fdata()

        # Normalize & Fix Labels
        X_vol = normalize(X_vol)
        y_vol[y_vol == 4] = 3
        y_vol = y_vol.astype(np.uint8)

        # Slice and Save
        n_slices = X_vol.shape[2]
        for i in range(0, n_slices, SLICE_DECIMATION):
            # Only save slices with actual brain/tumor data
            if np.sum(X_vol[:,:,i,0] > 0) > 500:
                
                # Resize
                img_slice = cv2.resize(X_vol[:, :, i, :], (IMG_SIZE, IMG_SIZE))
                # Nearest neighbor for masks to preserve integer labels
                mask_slice = cv2.resize(y_vol[:, :, i], (IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_NEAREST)
                
                # Save as compressed NumPy files
                # float16 saves 50% storage space
                save_name = f"{case_id}_{i}"
                np.save(f'{SAVE_DIR}/images/{save_name}.npy', img_slice.astype(np.float16))
                np.save(f'{SAVE_DIR}/masks/{save_name}.npy', mask_slice)

    except Exception as e:
        print(f"Skipping {case_id}: {e}")

# --- Run Preprocessing ---
patients = [d for d in os.listdir(TRAIN_DIR) if os.path.isdir(os.path.join(TRAIN_DIR, d))]
print(f"Preprocessing {len(patients)} patients...")

for patient_id in tqdm(patients):
    process_and_save_patient(os.path.join(TRAIN_DIR, patient_id), patient_id)

print("Processing Done! Data saved to /kaggle/working/processed_data")

Preprocessing 917 patients...


  0%|          | 1/917 [00:01<26:42,  1.75s/it]

Skipping BraTS2021_00656: Empty file: '/kaggle/input/instant-odc-ai-hackathon/Train/BraTS2021_00656/BraTS2021_00656_flair.nii'
Skipping BraTS2021_00466: Empty file: '/kaggle/input/instant-odc-ai-hackathon/Train/BraTS2021_00466/BraTS2021_00466_flair.nii'


  0%|          | 4/917 [00:02<06:10,  2.46it/s]

Skipping BraTS2021_01070: Empty file: '/kaggle/input/instant-odc-ai-hackathon/Train/BraTS2021_01070/BraTS2021_01070_seg.nii'


...

Skipping BraTS2021_00351: Empty file: '/kaggle/input/instant-odc-ai-hackathon/Train/BraTS2021_00351/BraTS2021_00351_seg.nii'
Skipping BraTS2021_00747: Empty file: '/kaggle/input/instant-odc-ai-hackathon/Train/BraTS2021_00747/BraTS2021_00747_flair.nii'


 99%|█████████▉| 912/917 [16:45<00:06,  1.26s/it]

Skipping BraTS2021_00683: Empty file: '/kaggle/input/instant-odc-ai-hackathon/Train/BraTS2021_00683/BraTS2021_00683_flair.nii'


100%|██████████| 917/917 [16:53<00:00,  1.11s/it]

Processing Done! Data saved to /kaggle/working/processed_data


In [25]:
class FastDataGenerator(Sequence):
    def __init__(self, file_list, batch_size=32, img_size=128, n_classes=4, shuffle=True):
        self.file_list = file_list
        self.batch_size = batch_size
        self.img_size = img_size
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.base_dir = '/kaggle/working/processed_data'
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.file_list) / self.batch_size))

    def __getitem__(self, index):
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        list_IDs_temp = [self.file_list[k] for k in indexes]
        return self.__data_generation(list_IDs_temp)

    def on_epoch_end(self):
        self.indexes = np.arange(len(self.file_list))
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        X = np.empty((self.batch_size, self.img_size, self.img_size, 4), dtype=np.float32)
        y = np.empty((self.batch_size, self.img_size, self.img_size, self.n_classes), dtype=np.float32)

        for i, ID in enumerate(list_IDs_temp):
            # Load from disk
            X[i,] = np.load(f'{self.base_dir}/images/{ID}.npy').astype(np.float32)
            mask = np.load(f'{self.base_dir}/masks/{ID}.npy')
            
            # One-hot encode
            y[i,] = tf.keras.utils.to_categorical(mask, num_classes=self.n_classes)

        return X, y

# --- Setup Generators ---
all_files = [f.replace('.npy', '') for f in os.listdir(f'{SAVE_DIR}/images')]
np.random.shuffle(all_files)

val_split = int(len(all_files) * 0.9) # 90% Training, 10% Validation
train_files = all_files[:val_split]
val_files = all_files[val_split:]

print(f"Training on {len(train_files)} slices")
print(f"Validating on {len(val_files)} slices")

train_gen = FastDataGenerator(train_files, batch_size=BATCH_SIZE)
val_gen = FastDataGenerator(val_files, batch_size=BATCH_SIZE)

Training on 11448 slices
Validating on 1272 slices


In [26]:
from tensorflow.keras import mixed_precision

# 1. Enable Mixed Precision (Speed Boost)
mixed_precision.set_global_policy('mixed_float16')

# 2. Define Model Components
def attention_gate(F_g, F_l, inter_filters):
    g = Conv2D(inter_filters, 1, padding='same')(F_g)
    x = Conv2D(inter_filters, 1, padding='same')(F_l)
    psi = Add()([g, x])
    psi = Activation('relu')(psi)
    psi = Conv2D(1, 1, padding='same')(psi)
    psi = Activation('sigmoid')(psi)
    return Multiply()([F_l, psi])

def conv_block(x, filters):
    x = Conv2D(filters, 3, padding='same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(filters, 3, padding='same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

def attention_unet(input_size, n_classes):
    inputs = Input(input_size)
    
    # Encoder
    c1 = conv_block(inputs, 64)
    p1 = MaxPooling2D((2, 2))(c1)
    c2 = conv_block(p1, 128)
    p2 = MaxPooling2D((2, 2))(c2)
    c3 = conv_block(p2, 256)
    p3 = MaxPooling2D((2, 2))(c3)
    c4 = conv_block(p3, 512)
    drop4 = Dropout(0.3)(c4)
    p4 = MaxPooling2D((2, 2))(drop4)

    # Bridge
    c5 = conv_block(p4, 1024)
    drop5 = Dropout(0.4)(c5)

    # Decoder
    u6 = UpSampling2D((2, 2))(drop5)
    u6 = Conv2D(512, 2, padding='same', kernel_initializer='he_normal')(u6)
    att6 = attention_gate(F_g=u6, F_l=drop4, inter_filters=256)
    merge6 = concatenate([att6, u6])
    c6 = conv_block(merge6, 512)

    u7 = UpSampling2D((2, 2))(c6)
    u7 = Conv2D(256, 2, padding='same', kernel_initializer='he_normal')(u7)
    att7 = attention_gate(F_g=u7, F_l=c3, inter_filters=128)
    merge7 = concatenate([att7, u7])
    c7 = conv_block(merge7, 256)

    u8 = UpSampling2D((2, 2))(c7)
    u8 = Conv2D(128, 2, padding='same', kernel_initializer='he_normal')(u8)
    att8 = attention_gate(F_g=u8, F_l=c2, inter_filters=64)
    merge8 = concatenate([att8, u8])
    c8 = conv_block(merge8, 128)

    u9 = UpSampling2D((2, 2))(c8)
    u9 = Conv2D(64, 2, padding='same', kernel_initializer='he_normal')(u9)
    att9 = attention_gate(F_g=u9, F_l=c1, inter_filters=32)
    merge9 = concatenate([att9, u9])
    c9 = conv_block(merge9, 64)

    # Output (Must be float32 for Mixed Precision stability)
    outputs = Conv2D(n_classes, 1)(c9)
    outputs = Activation('softmax', dtype='float32')(outputs)
    
    return Model(inputs, outputs)

# 3. Loss Functions
def dice_coef(y_true, y_pred, smooth=1e-6):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)

# 4. Compile
model = attention_unet((IMG_SIZE, IMG_SIZE, 4), N_CLASSES)
model.compile(optimizer=Adam(learning_rate=1e-3), loss=dice_loss, metrics=['accuracy', dice_coef])

In [27]:
checkpoint = ModelCheckpoint('model_2d.keras', monitor='val_dice_coef', mode='max', save_best_only=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6, verbose=1)

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=15, 
    callbacks=[checkpoint, reduce_lr]
)

Epoch 1/15
178/178 ━━━━━━━━━━━━━━━━━━━━ 0s 481ms/step - accuracy: 0.8677 - dice_coef: 0.7968 - loss: 0.2032
Epoch 1: val_dice_coef improved from -inf to 0.97336, saving model to model_2d.keras
178/178 ━━━━━━━━━━━━━━━━━━━━ 214s 537ms/step - accuracy: 0.8681 - dice_coef: 0.7975 - loss: 0.2025 - val_accuracy: 0.9844 - val_dice_coef: 0.9734 - val_loss: 0.0266 - learning_rate: 0.0010
Epoch 2/15
178/178 ━━━━━━━━━━━━━━━━━━━━ 0s 482ms/step - accuracy: 0.9900 - dice_coef: 0.9862 - loss: 0.0138
Epoch 2: val_dice_coef improved from 0.97336 to 0.98637, saving model to model_2d.keras
178/178 ━━━━━━━━━━━━━━━━━━━━ 92s 518ms/step - accuracy: 0.9900 - dice_coef: 0.9862 - loss: 0.0138 - val_accuracy: 0.9889 - val_dice_coef: 0.9864 - val_loss: 0.0136 - learning_rate: 0.0010
Epoch 3/15
178/178 ━━━━━━━━━━━━━━━━━━━━ 0s 481ms/step - accuracy: 0.9912 - dice_coef: 0.9898 - loss: 0.0102
Epoch 3: val_dice_coef improved from 0.98637 to 0.98973, saving model to model_2d.keras
178/178 ━━━━━━━━━━━━━━━━━━━━ 92s 518ms

In [28]:
# Function for RLE Encoding
def rle_encode(mask):
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

# Prediction Logic
def predict_and_submit(model, test_dir):
    test_cases = [d for d in os.listdir(test_dir) if os.path.isdir(os.path.join(test_dir, d))]
    submission_rows = []
    
    print(f"Predicting on {len(test_cases)} test cases...")
    
    for case_id in tqdm(test_cases):
        case_path = os.path.join(test_dir, case_id)
        
        # Load
        def get_path(pattern):
            matches = glob.glob(os.path.join(case_path, f"*{pattern}*"))
            return matches[0] if matches else None
            
        paths = [get_path("flair"), get_path("t1ce"), get_path("t1"), get_path("t2")]
        if not all(paths): continue
            
        # Stack & Normalize
        X_vol = np.stack([nib.load(p).get_fdata() for p in paths], axis=-1)
        orig_h, orig_w, orig_d = X_vol.shape[:3]
        X_vol = normalize(X_vol)
        
        # Predict Slice by Slice
        pred_vol = np.zeros((orig_h, orig_w, orig_d), dtype=np.uint8)
        
        for i in range(orig_d):
            # 1. Extract & Resize
            slice_img = cv2.resize(X_vol[:, :, i, :], (IMG_SIZE, IMG_SIZE))
            
            # 2. Predict (Add batch dim)
            slice_pred = model.predict(slice_img[np.newaxis, ...], verbose=0)[0]
            
            # 3. Argmax & Resize back to original
            mask_small = np.argmax(slice_pred, axis=-1)
            mask_orig = cv2.resize(mask_small.astype(float), (orig_w, orig_h), interpolation=cv2.INTER_NEAREST)
            
            pred_vol[:, :, i] = mask_orig
            
        # Create RLE for 3 classes
        # Model 1 -> Necrotic (Sub 1)
        # Model 2 -> Edema (Sub 2)
        # Model 3 -> Enhancing (Sub 4)
        
        rle_1 = rle_encode((pred_vol == 1).astype(np.uint8))
        rle_2 = rle_encode((pred_vol == 2).astype(np.uint8))
        rle_4 = rle_encode((pred_vol == 3).astype(np.uint8))
        
        submission_rows.append([f"{case_id}_1", rle_1])
        submission_rows.append([f"{case_id}_2", rle_2])
        submission_rows.append([f"{case_id}_4", rle_4])

    # Save
    df = pd.DataFrame(submission_rows, columns=['id', 'rle'])
    df.to_csv('submission.csv', index=False)
    print("Submission saved!")

# Run prediction
# Make sure to load best model first if training finished
# model.load_weights('model_2d.keras') 
predict_and_submit(model, TEST_DIR)

Predicting on 334 test cases...


100%|██████████| 334/334 [1:14:23<00:00, 13.37s/it]


Submission saved!
